In [22]:
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  

import tools.Sample_Tools as smpl
import tools.Pretreat_Tools as pretreat
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)

from base.JuUnits import parallal_task,task_chunk_split
from base.JuUnits import excute_for_multidates


from QUANTAXIS.QAUtil import DATABASE
from QUANTAXIS.QAUtil import  trade_date_sse
from QUANTAXIS.QAUtil.QADate_trade import (
    QA_util_get_pre_trade_date,
    QA_util_get_next_trade_date,
    QA_util_if_tradetime
)

import ind.alpha191 as a191
from scipy import stats
import statsmodels.api as sm

# import cpuinfo
# if 'ntel' in cpuinfo.get_cpu_info()['brand_raw']:
# from sklearnex import patch_sklearn, unpatch_sklearn
# unpatch_sklearn() ##注意，少量数据的线性回归没有优势。慎用，存在内存泄露

from sklearn import linear_model


%load_ext autoreload
%autoreload 2
%aimport tools.Cacher


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [135]:
def pretreate_data(data):
    returns = smpl.get_current_return(data,'close')
    returns.name = 'returns'
    ret_forward = smpl.get_forward_return(data,'close')
    ret_forward.name = 'ret_forward'
    # {'Open', 'cap', 'close', 'high', 'ind', 'low', 'returns', 'volume', 'vwap'}
    data = pd.concat([data, returns, ret_forward], axis=1)
    data = data.assign(vwap=data.amount/(data.volume*100))
    data = smpl.add_marketvalue_industry(data)
    data.rename(columns = {"open":"Open",'totalCapital':'cap','industry':'ind'}, inplace=True)
    data = smpl.add_report_inds(data,'netAssetsPerShare')
    data['pb']=data['close']/data['netAssetsPerShare']
    
    df_ids = df_treated.index.get_level_values(0)
    
    benchmark = smpl.get_benchmark('沪深300', start=df_ids.min(), end=df_ids.max()).data
    benchmark = benchmark[['open','close']].reset_index('code',drop=True)
    data['bm_index_open'] = df_ids.map(benchmark['open'])
    data['bm_index_close'] = df_ids.map(benchmark['close'])
    return data
    # return pd.concat([data, close_ind, vwap_ind, low_ind, high_ind, volume_ind, adv20_ind, adv40_ind, adv81_ind, co_mixed_ind, oh_mixed_ind, lv_mixed_ind], axis=1)


In [28]:
def assemble_stocks_by_codes(codes):
    files = list(map(lambda x:x+'_train_qfq',l))
    return pd.concat(list(map(lambda file:load_cache(file,cache_type=CACHE_TYPE.STOCK),files))).sort_index(level=0)

l = smpl.get_codes_from_blockname('沪深300', sse='all')
df_all =  assemble_stocks_by_codes(l)

In [136]:
df_treated = pretreate_data(df_all)

In [20]:
# ids = [11,24,38,41,42,47,57,69,80,82,83,88,93,97]
ids = [11,24]

# ids = np.arange(1,101)
# np.random.shuffle(ids)

# def generate_alpha_factors(fun_ids, df=None):
#     import ind.alpha191 as a191
#     from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)
#     from base.JuUnits import excute_for_multidates

# for i in fun_ids:
for i in ids:
    fun_name = 'alpha'+ '0'*(3- int(np.log10(i)+1)) +str(i)
    # indx = excute_for_multidates(df, lambda x: getattr(a191, fun_name)(x) ,level=1)
    indx = excute_for_multidates(df_treated, lambda x: getattr(a191, fun_name)(x) ,level=1)
    indx.name = fun_name
    # save_cache('{}_train'.format(fun_name), indx, cache_type=CACHE_TYPE.FACTOR)
    print(indx)
# worker=6
# task = task_chunk_split(ids, worker)
# results = parallal_task(worker, generate_alpha_factors, task, df=df_treated)


code
000001   -4.506804e+07
000002    1.596329e+07
000063   -3.630724e+07
000066    3.796777e+06
000069    2.500154e+07
              ...     
603882   -1.483158e+05
603899   -2.834211e+06
603986    1.431323e+05
603993   -1.358729e+08
605499   -4.952623e+05
Name: alpha011, Length: 248, dtype: float64
code
000001   -0.406365
000002   -0.211400
000063    0.276363
000066    0.133742
000069    0.348468
            ...   
603882    1.635349
603899    1.838936
603986    0.895157
603993   -0.114700
605499   -1.127755
Name: alpha024, Length: 248, dtype: float64


In [165]:
# excute_for_multidates(df_treated.sort_index(), lambda x: a191.alpha011(x) ,level=1)
def STD(data, windows):
    return data.rolling(window=windows, min_periods=windows).std()
def MEAN(data, windows):
    return data.rolling(window=windows, min_periods=windows).mean()
def DELTA(data, windows):
    return data.diff(4)
def SEQUENCE(n):
    return pd.Series(np.arange(1,n+1))

def SMA(data,windows,alpha):
    return data.ewm(adjust=False, alpha=float(alpha)/windows, min_periods=windows, ignore_na=False).mean()

def REGBETA(xs, y, n):
    assert len(y)>=n,  'len(y)!>=n !!!'
    regress = linear_model.LinearRegression(fit_intercept=False)
    def reg(X,Y):
        try:
            if len(Y)>len(X):
                Y_ =  Y[X.index]
                if Y_.isnull().any():
                    return np.nan
                res = regress.fit(X.values.reshape(-1, 1), Y_.values.reshape(-1, 1)).coef_[0]
            else:
                # if Y.isnull().any():
                #     return np.nan
                res = regress.fit(X.values.reshape(-1, 1), Y.values.reshape(-1, 1)).coef_[0]
        except Exception as e:
            print(e)
            return np.nan
        return res
    return xs.rolling(window=n, min_periods=n).apply(lambda x:reg(x,y))


def COVIANCE(A,B,d):
    se = pd.Series(np.arange(len(A.index)),index=A.index)
    se = se.rolling(5).apply(lambda x: A.iloc[x].cov(B.iloc[x]))
    return se

def CORR(A,B,d):
    se = pd.Series(np.arange(len(A.index)),index=A.index)
    se = se.rolling(5).apply(lambda x: A.iloc[x].corr(B.iloc[x]))
    return se


def alpha181(data, dependencies=['close'], max_window=40):
    # SUM(RET-MEAN(RET,20)-(BANCHMARK_INDEX_CLOSE-MEAN(BANCHMARK_INDEX_CLOSE,20))^2,20)/SUM((BANCHMARK_INDEX_CLOSE-MEAN(BANCHMARK_INDEX_CLOSE,20))^3)
    # BANCHMARK_INDEX_CLOSE
    bm = data['bm_index_close']
    bm_mean = bm - bm.rolling(window=20, min_periods=20).mean()
    # bm_mean = pd.DataFrame(data=np.repeat(bm_mean.values.reshape(len(bm_mean.values),1), len(data['close'].columns), axis=1), index=data['close'].index, columns=data['close'].columns)
    ret = data['close'].pct_change(periods=1)
    part1 = (ret-ret.rolling(window=20,min_periods=20).mean()-bm_mean**2).rolling(window=20,min_periods=20).sum()
    part2 = (bm_mean ** 3).rolling(window=20,min_periods=20).sum()
    return part1 / part2

print(alpha181(df_treated.loc[(slice(None),['000001']),:]).to_list())


[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, -0.014515493237541727, -0.014383483200177696, -0.014349568029882503, -0.01433195460703183, -0.014485003098385177, -0.014538718532256694, -0.014330998369566417, -0.014399014038354625, -0.014159213879189718, -0.014858761401507655, -0.01687441749146269, -0.023658772037964886, -0.0313452552579826, -0.05096076326115327, -0.05142837047528231, -0.055088438905750964, -0.06227516758680097, -0.07271653099160258, -0.07162100637318862, -0.044126190549397835, -0.032352379614622014, -0.028222746204232204, -0.02678942529368811, -0.02513083676639192, -0.025358326152140277, -0.025533525289817467, -0.025442955016801903, -0.02507883945204089, -0.025723963431121138, -0.02991031221246133, -0.03084108441013144, -0.031450536626231373, -0.04539863517313729, -0.05926406849099986, -0.06970805036257487, -0.0790098327260

In [21]:
# a = df_treated.loc[(slice(None),['000001']),:]
alpha021(a)

date        code  
2017-01-04  000001         NaN
2017-01-05  000001         NaN
2017-01-06  000001         NaN
2017-01-09  000001         NaN
2017-01-10  000001         NaN
                        ...   
2021-12-24  000001    0.201586
2021-12-27  000001    0.202608
2021-12-28  000001    0.203365
2021-12-29  000001    0.203962
2021-12-30  000001    0.204713
Name: close, Length: 1215, dtype: float64

In [93]:
%%time
# df_all.loc[(slice('2017-01-10'),['000001','000002','000063']),:]
# pd.concat(indxes,axis=1).sort_index().loc[(slice('2017-01-15'),['000001','000002','000063']),:]

# # a = load_cache('alpha11_train', cache_type=CACHE_TYPE.FACTOR)
# # b = load_cache('alpha24_train', cache_type=CACHE_TYPE.FACTOR)
# pd.concat([a,b],axis=1).sort_index().loc[(slice('2017-01-20'),['000001','000002']),:]
# pd.Series(-1,index=df_treated.loc[(slice(None),['000001']),:].index,dtype=np.dtype('int8'))

# a=alpha004(df_treated.loc[(slice(None),['000001']),:])
# a.rolling(window=5, min_periods=5).corr(a).to_list()
# a.to_list()
# a.rolling(window=5, min_periods=5).apply(lambda x:x.corr(a[x.index])).to_list()
# a[0:50].rolling(window=5, min_periods=5).apply(lambda x:print(x.corr(a[x.index])))


# def alpha083(data, dependencies=['high', 'volume'], max_window=5):
#     # (-1*RANK(COVIANCE(RANK(HIGH),RANK(VOLUME),5)))
#     alpha = COVIANCE(data['high'].rank(pct=True),data['volume'].rank(pct=True),5)*-1
#     return alpha

# print(alpha083(df_treated.loc[(slice(None),['000001']),:]).to_list()[0:100])

# import matplotlib.pyplot as plt
# plt.plot(np.arange(100),alpha083(df_treated.loc[(slice(None),['000001']),:]).to_list()[0:100])
# plt.show()
# plt.plot(np.arange(100),alpha090(df_treated.loc[(slice(None),['000001']),:]).to_list()[0:100])
# plt.show()


# X = df_treated.loc[(slice(None),['000001']),:]['close']
# Y = pd.Series(np.random.rand(1215),index=X.index)
# X = pd.Series(np.random.rand(10000000))
# Y = pd.Series(np.random.rand(10000000))

# x=np.array(range(1, 10))
# w = np.array(range(1, 10))


Wall time: 0 ns


In [26]:
%%time
# b=akshare.stock_zh_a_tick_163(code="sz003023", trade_date="20221025") 
# c = akshare.stock_zh_a_tick_tx_js(code="sz003023")
# c['成交时间']
# b['时间']

# X2=X.to_numpy().reshape(-1, 1)
# Y2=Y.to_numpy().reshape(-1, 1)

# reg = linear_model.LinearRegression(fit_intercept=False)
# reg.fit(X2,Y2).coef_

# a = df_treated.loc[(slice(None),['000001']),:]
# x = a['close']
# y = a['Open']




Wall time: 25.2 ms


,,open,close,high,low,vol,amount,up_count,down_count,date_stamp,volume
date,code,,,,,,,,,,
2017-01-04,000300,3340.56,3368.31,3369.57,3338.15,900508.0,9.801436e+10,248,28,1.483459e+09,900508.0
2017-01-05,000300,3368.34,3367.79,3373.41,3361.46,883363.0,9.118934e+10,120,150,1.483546e+09,883363.0
2017-01-06,000300,3366.85,3347.67,3368.87,3346.89,874591.0,9.107226e+10,69,208,1.483632e+09,874591.0
2017-01-09,000300,3345.75,3363.90,3366.22,3344.35,837079.0,8.791612e+10,213,65,1.483891e+09,837079.0
2017-01-10,000300,3361.64,3358.27,3370.51,3354.09,838191.0,8.253397e+10,103,172,1.483978e+09,838191.0
...,...,...,...,...,...,...,...,...,...,...,...
2021-12-24,000300,4953.06,4921.34,4964.91,4905.98,1426733.0,2.904837e+11,116,178,1.640275e+09,1426733.0
2021-12-27,000300,4918.15,4919.32,4939.69,4894.16,1241009.0,2.544215e+11,178,112,1.640534e+09,1241009.0
2021-12-28,000300,4923.69,4955.96,4959.15,4913.60,1304577.0,2.763167e+11,179,114,1.640621e+09,1304577.0


In [132]:
# test = df_treated
# bm = smpl.get_benchmark('沪深300', start=test.index.get_level_values(0).min(), end=test.index.get_level_values(0).max()).data
# bm_o = bm[['open','close']].reset_index('code',drop=True)

test['index_open'] = test.index.get_level_values(0).map(bm_o['open'])
test['index_close'] = test.index.get_level_values(0).map(bm_o['close'])
test.iloc[:,-3:].sort_index()

new  index_open  index_close
date       code                                    
2017-01-04 000001  3340.56     3340.56      3368.31
           000002  3340.56     3340.56      3368.31
           000063  3340.56     3340.56      3368.31
           000066  3340.56     3340.56      3368.31
           000069  3340.56     3340.56      3368.31
...                    ...         ...          ...
2021-12-30 603882  4882.16     4882.16      4921.51
           603899  4882.16     4882.16      4921.51
           603986  4882.16     4882.16      4921.51
           603993  4882.16     4882.16      4921.51
           605499  4882.16     4882.16      4921.51

[276463 rows x 3 columns]

In [115]:
# test.index.get_level_values(0).map(bm_o)
bm_o

date
2017-01-04    3340.56
2017-01-05    3368.34
2017-01-06    3366.85
2017-01-09    3345.75
2017-01-10    3361.64
               ...   
2021-12-24    4953.06
2021-12-27    4918.15
2021-12-28    4923.69
2021-12-29    4958.65
2021-12-30    4882.16
Name: bm_o, Length: 1215, dtype: float64